In [1]:
import csv

# Open the text file for reading
with open('MU.txt', 'r') as text_file:
    # Read each line of the file
    lines = text_file.readlines()

# Initialize an empty list to store the rows of data
data = []

# Process each line
for line in lines:
    # Split the line by spaces
    values = line.strip().split('\t')
    # Append the values as a list to the data list
    data.append(values)


# Specify the name of the CSV file
csv_file_name = 'output.csv'

# Write the data to a CSV file
with open(csv_file_name, 'w', newline='') as csv_file:
    csv_writer = csv.writer(csv_file)
    # Write the header if needed
    # csv_writer.writerow(['Column1', 'Column2', ...])
    # Write the rows of data
    csv_writer.writerows(data)

print(f'Data has been successfully converted to CSV and saved as {csv_file_name}.')

Data has been successfully converted to CSV and saved as output.csv.


In [2]:
import pandas as pd
import numpy as np
df = pd.read_csv("output.csv", header=None)
df.columns = ['id', 'event', 'device', 'channel', 'digit', 'size', 'raw_data']
df

,id,event,device,channel,digit,size,raw_data
0,978111,132669,MU,TP9,6,459,"475,474,477,486,486,476,479,483,489,483,482,48..."
1,978112,132669,MU,FP1,6,459,"468,487,493,493,498,493,491,490,492,487,483,48..."
2,978113,132669,MU,FP2,6,459,"482,475,490,500,485,470,470,482,490,484,478,48..."
3,978114,132669,MU,TP10,6,459,"470,470,478,489,487,475,469,478,488,483,474,48..."
4,978115,132670,MU,TP9,7,493,"506,499,495,491,492,507,496,500,498,496,499,50..."
...,...,...,...,...,...,...,...
163927,1142038,173650,MU,TP10,-1,460,"537,531,530,531,533,537,537,555,534,522,531,54..."
163928,1142039,173651,MU,TP9,-1,460,"521,517,526,524,514,519,520,526,526,514,519,52..."
163929,1142040,173651,MU,FP1,-1,460,"525,518,522,525,523,523,512,523,519,522,523,52..."
163930,1142041,173651,MU,FP2,-1,460,"502,493,500,511,510,512,523,507,497,503,502,50..."


In [3]:
data = df['raw_data'].tolist()
parsed_data = []
for line in data:
    tmp = line.split(',')
    parsed_data.append(np.array(tmp))
df.drop('raw_data', axis=1, inplace=True)
df['data'] = parsed_data
df

,id,event,device,channel,digit,size,data
0,978111,132669,MU,TP9,6,459,"[475, 474, 477, 486, 486, 476, 479, 483, 489, ..."
1,978112,132669,MU,FP1,6,459,"[468, 487, 493, 493, 498, 493, 491, 490, 492, ..."
2,978113,132669,MU,FP2,6,459,"[482, 475, 490, 500, 485, 470, 470, 482, 490, ..."
3,978114,132669,MU,TP10,6,459,"[470, 470, 478, 489, 487, 475, 469, 478, 488, ..."
4,978115,132670,MU,TP9,7,493,"[506, 499, 495, 491, 492, 507, 496, 500, 498, ..."
...,...,...,...,...,...,...,...
163927,1142038,173650,MU,TP10,-1,460,"[537, 531, 530, 531, 533, 537, 537, 555, 534, ..."
163928,1142039,173651,MU,TP9,-1,460,"[521, 517, 526, 524, 514, 519, 520, 526, 526, ..."
163929,1142040,173651,MU,FP1,-1,460,"[525, 518, 522, 525, 523, 523, 512, 523, 519, ..."
163930,1142041,173651,MU,FP2,-1,460,"[502, 493, 500, 511, 510, 512, 523, 507, 497, ..."


In [4]:
# raw data
all_events = np.unique(df['event'])
arrays = []
digit_labels = []
data_size = 96*4
for event in all_events:
    tmp_df = df.query(f"event=={event}")[['channel','data','digit','size']].set_index('channel')
    digit_labels.append(np.unique(tmp_df['digit']).item())

    # print(tmp_df)

    this_size = tmp_df.loc['TP9',:]['size']
    if this_size >= data_size:
        tmp_tp9 = tmp_df.loc['TP9',:]['data'][0:data_size].astype(float)
        tmp_tp10 = tmp_df.loc['TP10',:]['data'][0:data_size].astype(float)
        tmp_fp1 = tmp_df.loc['FP1',:]['data'][0:data_size].astype(float)
        tmp_fp2 = tmp_df.loc['FP2',:]['data'][0:data_size].astype(float)
    else:
        tmp_tp9 = np.zeros(data_size)
        tmp_tp9[0:this_size] = tmp_df.loc['TP9',:]['data'].astype(float)
        tmp_tp10 = np.zeros(data_size)
        tmp_tp10[0:this_size] = tmp_df.loc['TP10',:]['data'].astype(float)
        tmp_fp1 = np.zeros(data_size)
        tmp_fp1[0:this_size] = tmp_df.loc['FP1',:]['data'].astype(float)
        tmp_fp2 = np.zeros(data_size)
        tmp_fp2[0:this_size] = tmp_df.loc['FP2',:]['data'].astype(float)

    # tmp_array = np.column_stack((tmp_tp9,tmp_tp10,tmp_fp1,tmp_fp2))[0:min_size, :]
    tmp_array = np.row_stack((tmp_tp9,tmp_tp10,tmp_fp1,tmp_fp2))
    
    # result = np.fft.fft(tmp_array)
    arrays.append(tmp_array)

In [5]:
# # normalize data
# from scipy.stats import zscore

# all_events = np.unique(df['event'])
# z_arrays = []
# digit_labels = []
# data_size = 480
# for event in all_events:
#     tmp_df = df.query(f"event=={event}")[['channel','data','digit','size']].set_index('channel')
#     digit_labels.append(np.unique(tmp_df['digit']).item())

#     # print(tmp_df)

#     this_size = tmp_df.loc['TP9',:]['size']
#     if this_size >= data_size:
#         tmp_tp9 = zscore(tmp_df.loc['TP9',:]['data'][0:data_size].astype(float))
#         tmp_tp10 = zscore(tmp_df.loc['TP10',:]['data'][0:data_size].astype(float))
#         tmp_fp1 = zscore(tmp_df.loc['FP1',:]['data'][0:data_size].astype(float))
#         tmp_fp2 = zscore(tmp_df.loc['FP2',:]['data'][0:data_size].astype(float))
#     else:
#         tmp_tp9 = np.zeros(data_size)
#         tmp_tp9[0:this_size] = zscore(tmp_df.loc['TP9',:]['data'].astype(float))
#         tmp_tp10 = np.zeros(data_size)
#         tmp_tp10[0:this_size] = zscore(tmp_df.loc['TP10',:]['data'].astype(float))
#         tmp_fp1 = np.zeros(data_size)
#         tmp_fp1[0:this_size] = zscore(tmp_df.loc['FP1',:]['data'].astype(float))
#         tmp_fp2 = np.zeros(data_size)
#         tmp_fp2[0:this_size] = zscore(tmp_df.loc['FP2',:]['data'].astype(float))

#     # tmp_array = np.column_stack((tmp_tp9,tmp_tp10,tmp_fp1,tmp_fp2))[0:min_size, :]
#     tmp_array = np.row_stack((tmp_tp9,tmp_tp10,tmp_fp1,tmp_fp2))
    
#     # result = np.fft.fft(tmp_array)
#     z_arrays.append(tmp_array)

In [6]:
# # normalize data
# def my_norm(a):
#     ratio = 2/(np.max(a)-np.min(a)) 
#     shift = (np.max(a)+np.min(a))/2 
#     return (a - shift)*ratio

# all_events = np.unique(df['event'])
# z_arrays = []
# digit_labels = []
# data_size = 480
# for event in all_events:
#     tmp_df = df.query(f"event=={event}")[['channel','data','digit','size']].set_index('channel')
#     digit_labels.append(np.unique(tmp_df['digit']).item())

#     # print(tmp_df)

#     this_size = tmp_df.loc['TP9',:]['size']
#     if this_size >= data_size:
#         tmp_tp9 = my_norm(tmp_df.loc['TP9',:]['data'][0:data_size].astype(float))
#         tmp_tp10 = my_norm(tmp_df.loc['TP10',:]['data'][0:data_size].astype(float))
#         tmp_fp1 = my_norm(tmp_df.loc['FP1',:]['data'][0:data_size].astype(float))
#         tmp_fp2 = my_norm(tmp_df.loc['FP2',:]['data'][0:data_size].astype(float))
#     else:
#         tmp_tp9 = np.zeros(data_size)
#         tmp_tp9[0:this_size] = my_norm(tmp_df.loc['TP9',:]['data'].astype(float))
#         tmp_tp10 = np.zeros(data_size)
#         tmp_tp10[0:this_size] = my_norm(tmp_df.loc['TP10',:]['data'].astype(float))
#         tmp_fp1 = np.zeros(data_size)
#         tmp_fp1[0:this_size] = my_norm(tmp_df.loc['FP1',:]['data'].astype(float))
#         tmp_fp2 = np.zeros(data_size)
#         tmp_fp2[0:this_size] = my_norm(tmp_df.loc['FP2',:]['data'].astype(float))

#     tmp_array = np.row_stack((tmp_tp9,tmp_tp10,tmp_fp1,tmp_fp2))
#     z_arrays.append(tmp_array)

In [7]:
# normalize data
def my_norm(a, the_max, the_min):
    ratio = 2/(the_max-the_min) 
    shift = (the_max+the_min)/2 
    return (a - shift)*ratio

all_events = np.unique(df['event'])
z_arrays = []
digit_labels = []
data_size = 96*4
for event in all_events:
    tmp_df = df.query(f"event=={event}")[['channel','data','digit','size']].set_index('channel')
    digit_labels.append(np.unique(tmp_df['digit']).item())

    # print(tmp_df)
    this_data = np.row_stack(tmp_df['data'].to_numpy()).astype(float)
    this_size = (tmp_df.loc['TP9',:]['size'])
    the_max = np.max(this_data)
    the_min = np.min(this_data)
    if this_size >= data_size:
        tmp_tp9 = my_norm(tmp_df.loc['TP9',:]['data'][0:data_size].astype(float), the_max, the_min)
        tmp_tp10 = my_norm(tmp_df.loc['TP10',:]['data'][0:data_size].astype(float), the_max, the_min)
        tmp_fp1 = my_norm(tmp_df.loc['FP1',:]['data'][0:data_size].astype(float), the_max, the_min)
        tmp_fp2 = my_norm(tmp_df.loc['FP2',:]['data'][0:data_size].astype(float), the_max, the_min)
    else:
        tmp_tp9 = np.zeros(data_size)
        tmp_tp9[0:this_size] = my_norm(tmp_df.loc['TP9',:]['data'].astype(float), the_max, the_min)
        tmp_tp10 = np.zeros(data_size)
        tmp_tp10[0:this_size] = my_norm(tmp_df.loc['TP10',:]['data'].astype(float), the_max, the_min)
        tmp_fp1 = np.zeros(data_size)
        tmp_fp1[0:this_size] = my_norm(tmp_df.loc['FP1',:]['data'].astype(float), the_max, the_min)
        tmp_fp2 = np.zeros(data_size)
        tmp_fp2[0:this_size] = my_norm(tmp_df.loc['FP2',:]['data'].astype(float), the_max, the_min)

    tmp_array = np.row_stack((tmp_tp9,tmp_tp10,tmp_fp1,tmp_fp2))
    z_arrays.append(tmp_array)

In [17]:
arrays

[array([[475., 474., 477., ..., 518., 515., 514.],
        [470., 470., 478., ..., 517., 505., 506.],
        [468., 487., 493., ..., 510., 507., 511.],
        [482., 475., 490., ..., 496., 495., 493.]]),
 array([[506., 499., 495., ..., 517., 520., 517.],
        [500., 506., 490., ..., 508., 521., 511.],
        [505., 515., 513., ..., 516., 513., 513.],
        [511., 501., 503., ..., 512., 507., 510.]]),
 array([[520., 533., 528., ..., 504., 502., 494.],
        [526., 533., 531., ..., 493., 503., 506.],
        [502., 516., 514., ..., 510., 504., 502.],
        [514., 511., 514., ..., 499., 504., 504.]]),
 array([[512., 530., 527., ..., 498., 496., 496.],
        [517., 523., 531., ..., 503., 489., 489.],
        [512., 529., 527., ..., 509., 512., 511.],
        [517., 515., 518., ..., 505., 504., 499.]]),
 array([[502., 503., 498., ..., 516., 515., 509.],
        [501., 504., 494., ..., 515., 513., 505.],
        [506., 511., 513., ..., 509., 504., 508.],
        [513., 512., 50

In [9]:
z_arrays

[array([[-0.62025316, -0.64556962, -0.56962025, ...,  0.46835443,
          0.39240506,  0.36708861],
        [-0.74683544, -0.74683544, -0.5443038 , ...,  0.44303797,
          0.13924051,  0.16455696],
        [-0.79746835, -0.3164557 , -0.16455696, ...,  0.26582278,
          0.18987342,  0.29113924],
        [-0.44303797, -0.62025316, -0.24050633, ..., -0.08860759,
         -0.11392405, -0.16455696]]),
 array([[-0.1641791 , -0.37313433, -0.49253731, ...,  0.1641791 ,
          0.25373134,  0.1641791 ],
        [-0.34328358, -0.1641791 , -0.64179104, ..., -0.10447761,
          0.28358209, -0.01492537],
        [-0.19402985,  0.10447761,  0.04477612, ...,  0.13432836,
          0.04477612,  0.04477612],
        [-0.01492537, -0.31343284, -0.25373134, ...,  0.01492537,
         -0.13432836, -0.04477612]]),
 array([[ 0.37662338,  0.71428571,  0.58441558, ..., -0.03896104,
         -0.09090909, -0.2987013 ],
        [ 0.53246753,  0.71428571,  0.66233766, ..., -0.32467532,
         -0.

In [10]:
digit_labels

[6,
 7,
 6,
 6,
 1,
 6,
 6,
 8,
 1,
 9,
 3,
 7,
 1,
 9,
 2,
 0,
 5,
 9,
 6,
 5,
 5,
 2,
 9,
 8,
 1,
 0,
 3,
 2,
 9,
 5,
 8,
 8,
 7,
 7,
 9,
 2,
 1,
 0,
 9,
 2,
 5,
 3,
 2,
 6,
 2,
 1,
 1,
 1,
 0,
 7,
 0,
 0,
 8,
 2,
 5,
 9,
 4,
 0,
 7,
 1,
 9,
 1,
 0,
 6,
 2,
 8,
 6,
 9,
 8,
 1,
 2,
 3,
 8,
 6,
 8,
 6,
 9,
 5,
 9,
 2,
 7,
 0,
 4,
 7,
 4,
 8,
 2,
 1,
 0,
 7,
 5,
 6,
 1,
 9,
 7,
 9,
 8,
 5,
 9,
 6,
 5,
 5,
 4,
 1,
 7,
 7,
 2,
 4,
 3,
 5,
 4,
 3,
 9,
 5,
 3,
 8,
 7,
 5,
 6,
 8,
 0,
 7,
 6,
 2,
 9,
 7,
 9,
 9,
 7,
 8,
 8,
 9,
 9,
 0,
 1,
 1,
 0,
 7,
 1,
 6,
 3,
 0,
 0,
 3,
 5,
 7,
 3,
 3,
 5,
 0,
 4,
 0,
 9,
 0,
 1,
 5,
 7,
 7,
 9,
 8,
 5,
 6,
 6,
 2,
 7,
 9,
 4,
 1,
 6,
 9,
 5,
 2,
 5,
 5,
 0,
 2,
 8,
 1,
 3,
 2,
 3,
 5,
 5,
 7,
 9,
 6,
 9,
 3,
 8,
 6,
 1,
 5,
 6,
 8,
 8,
 9,
 8,
 3,
 8,
 9,
 5,
 4,
 8,
 4,
 6,
 6,
 8,
 3,
 9,
 2,
 8,
 1,
 5,
 1,
 9,
 4,
 0,
 3,
 5,
 3,
 3,
 5,
 0,
 6,
 8,
 6,
 3,
 0,
 7,
 2,
 3,
 9,
 4,
 8,
 4,
 4,
 7,
 8,
 0,
 7,
 3,
 0,
 9,
 3,
 5,
 2,
 4,
 9,
 9,
 6,


In [11]:
len(arrays)

40983

In [12]:
len(digit_labels)

40983

In [13]:
# save to pickle file
import pickle
with open('MindMNIST.pkl', 'wb') as f:
    pickle.dump(arrays, f)
    pickle.dump(z_arrays, f)
    pickle.dump(digit_labels, f)

In [14]:
# # load the pickle file
# with open('MindMINST.pkl', 'rb') as f:
#     arrays = pickle.load(f) # list of numpy arrays, each numpy array correspond to an event after fft
#     digit_labels = pickle.load(f) # list of digit labels corresponding to each event